<a href="https://colab.research.google.com/github/duartemiguel30/colab_tia/blob/main/C%C3%B3digo_funcional_novo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl

from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Caminho do seu arquivo JSONL
train_path = "/content/casos_convertidos_com_invalidos.jsonl"

# Carrega o dataset do JSONL
dataset = load_dataset("json", data_files=train_path, split="train")

# Define as labels manualmente (ordem importa!)
labels = ["amarela", "laranja", "caso inválido", "vermelha", "azul", "verde"]
class_label = ClassLabel(names=labels)

# Converte label textual para índice numérico
def converter_label(ex):
    ex["label_id"] = class_label.str2int(ex["labels"])  # usa coluna 'labels' que está no JSON
    return ex

dataset = dataset.map(converter_label)

# Tokenização com BERT português
modelo_nome = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo_nome)

def tokenizar(ex):
    return tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

dataset = dataset.map(tokenizar)

# Remove coluna textual 'labels' antes de renomear
dataset = dataset.remove_columns(["labels"])  # remove string
dataset = dataset.rename_column("label_id", "labels")  # renomeia id para 'labels' (agora numérico)

# Remove outras colunas desnecessárias (opcional, mas recomendado)
dataset = dataset.remove_columns(["text"])  # mantém só input_ids, attention_mask, labels

# Carrega modelo pré-treinado para classificação
modelo = AutoModelForSequenceClassification.from_pretrained(
    modelo_nome,
    num_labels=len(labels),
    id2label={i: l for i, l in enumerate(labels)},
    label2id={l: i for i, l in enumerate(labels)}
)

args = TrainingArguments(
    output_dir="./modelo_treinado",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    learning_rate=2e-5,
    logging_dir="./logs",
    report_to="none",
    fp16=True
)

# Inicializa o Trainer
trainer = Trainer(
    model=modelo,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

# Treina o modelo
trainer.train()

# Salva o modelo e o tokenizer
modelo.save_pretrained("./modelo_treinado")
tokenizer.save_pretrained("./modelo_treinado")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-17-fb29749ee7ed>:61: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


('./modelo_treinado/tokenizer_config.json',
 './modelo_treinado/special_tokens_map.json',
 './modelo_treinado/vocab.txt',
 './modelo_treinado/added_tokens.json',
 './modelo_treinado/tokenizer.json')

In [ ]:
from transformers import pipeline

# Caminho onde o modelo e o tokenizer foram salvos
modelo_path = "./modelo_treinado"

# Cria pipeline de classificação de texto
clf = pipeline("text-classification", model=modelo_path, tokenizer=modelo_path)

# Teste com um exemplo
texto = "Paciente com dores da cabeça ."
resultado = clf(texto)

print("Resultado:", resultado)


Device set to use cpu


Resultado: [{'label': 'verde', 'score': 0.5813533663749695}]
